# Cleaning
- renommer des colonnes (df.rename(columns=....))
- trouver/supprimer les données dupliquées (.duplicated / .drop_duplicates)
- trouver les NA (df.column.isna() / df.column.notna())
- remplacer les NA (.fillna())
- remplacer n'importe quelle valeur (.replace({OLD_VALUE: NEW_VALUE, …}))
- changer le type d'une série (aka cast) (.astype(type) / pd.to_numeric / pd.to_datetime)
- fallback les valeurs NA d'une colonne sur une autre colonne: combine_first

### .dt accessor (for date-type columns)
→ https://pandas.pydata.org/pandas-docs/stable/reference/series.html#datetime-properties

### .str accessor (for string-typed columns)

→ https://pandas.pydata.org/pandas-docs/stable/reference/series.html#string-handling

### Regexes:
- cheat sheet: https://www.debuggex.com/cheatsheet/regex/python
- talk sympa: https://www.youtube.com/watch?v=abrcJ9MpF60
- le module dédié "re" de python: https://docs.python.org/3/library/re.html
- site web pour tester des regexes: https://regex101.com

In [61]:
import pandas as pd
from datetime import datetime
import re

In [5]:
# if you don't want pandas.read_csv to mess with data types,
# you can force it to keep str values by specifying dtype=str.
people = pd.read_csv('./inputs/people.csv')

In [6]:
people.shape

(209, 15)

In [7]:
people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 15 columns):
id               209 non-null int64
first_name       207 non-null object
last_name        207 non-null object
email address    203 non-null object
gender           207 non-null object
age              207 non-null object
money            190 non-null object
lon              207 non-null float64
lat              207 non-null float64
phone            83 non-null object
registration     207 non-null object
inactive         207 non-null object
last_seen        190 non-null float64
address          207 non-null object
preference       207 non-null object
dtypes: float64(3), int64(1), object(11)
memory usage: 24.6+ KB


In [8]:
people.gender.unique()

array(['Female', 'Male', 'F', 'M', nan], dtype=object)

In [9]:
def clean_people(df):
    # rename columns:
    df = df.rename(columns={'email address': 'email'})
    
    # remove rows which have an empty "first_name" (NA):
    #df = df[df.first_name.notna()] <- equivalent to next line:
    df = df.dropna(subset=['first_name'])
    
    # drop duplicates on ID column:
    df = df.drop_duplicates()
    
    # Normalize gender column:
    df['gender'] = df['gender'].replace({'Female': 'F', 'Male': 'M'})
    
    # Convert column "age" to number (coerce: put NaN for bad values):
    df['age'] = pd.to_numeric(df.age, errors='coerce')
    
    # Convert columns to date type:
    df['registration'] = pd.to_datetime(df.registration)
    df['last_seen'] = pd.to_datetime(df.last_seen, unit='s')
    # When missing, last seen should fallback to the registration date:
    df['last_seen'] = df.last_seen.combine_first(df.registration)
    
    # Add a "full_name" column by concatenating two other ones:
    df['full_name'] = df.first_name + " " + df.last_name
    
    # Add a "country" column by extracting it from the address, with a split:
    df['country'] = df.address.str.split(', ').str[1]
    
    # Column "money" contains values like "$50.23" or "€23,09".
    # We want to make it uniform (only dollar currency) and as number, not str.
    df['currency'] = df.money.str[0]  # extract first char ($/€) to a new "currency" column
    df['money'] = df.money.str[1:].str.replace(',', '.')  # extract remaining chars and replace , by .
    df['money'] = pd.to_numeric(df.money)  # convert to number
    # convert euros cells to dollar:
    df.loc[df.currency == '€', 'money'] = df[df.currency == '€'].money * 1.10
    del df['currency']  # remove "currency" column which is now useless
    
    # Keep only rows where email is not NA:
    df = df.dropna(subset=['email'])
    # Keep only rows where email is a good email:
    # CAUTION: in the real world you should not use dummy regexes like this to validate email addresses,
    # but instead use a dedicated tool like https://github.com/syrusakbary/validate_email.
    df = df[df.email.str.contains('.+@[0-9a-zA-Z\.\-_]+\.\w{2,}')]
    # Some users may use email alias (example: john.smith+truc@gmail.com is an alias for john.smith@gmail.com).
    # We want to drop these duplicates. To do that, we extract the 'alias' part with a regex:
    groups = df.email.str.extract('([0-9a-zA-Z\.\-_]+)(\+[0-9a-zA-Z\.\-_]+)?(@[0-9a-zA-Z\.\-_]+\.\w{2,})')
    df['email'] = groups[0] + groups[2]  # we override the email with the email without the alias part
    # Then, just use drop_duplicates, which will keep the first line by default:
    df = df.drop_duplicates(subset=['email'])
    
    return df

df_clean = clean_people(people)

## Exo cours 5

### Mise à jour de la colonne inactive

In [84]:
is_inactive = df_clean.last_seen < datetime.now()
df["incative"] = is_inactive
df_clean.head(10)

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country,is_mobile_phone
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.180,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Czech Republic,None
1,27570,Ruthi,Ross,rross3w@sohu.com,F,57.0,20.370,19.230220,50.466575,None,2018-10-23,False,2019-08-30 11:41:04,"Siewierz, Poland",entrée/plat/dessert,Ruthi Ross,Poland,None
2,27572,Silas,Stourton,silas.stourton3y@answers.com,M,22.0,36.289,118.831081,24.984813,None,2018-12-30,False,2019-03-27 13:10:32,"Longbo, China",entrée/plat/dessert,Silas Stourton,China,None
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64.0,108.823,121.648987,-8.844744,None,2019-03-12,False,2019-03-12 00:00:00,"Potulando, Indonesia",entrée/plat,Roxine Pettecrew,Indonesia,None
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.260,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Czech Republic,None
6,27539,Dulcine,Stopforth,dstopforth31@forbes.com,F,22.0,13.860,-35.587670,-6.255393,None,2017-12-13,False,2019-01-15 15:39:05,"Serrinha, Brazil",entrée/plat/dessert,Dulcine Stopforth,Brazil,None
7,27469,Denny,Hasnip,dhasnip13@so-net.ne.jp,F,47.0,83.567,-89.917591,14.113818,None,2017-05-01,False,2019-03-08 18:46:46,"Comapa, Guatemala",entrée/plat/dessert,Denny Hasnip,Guatemala,None
9,27565,Esmeralda,Hollindale,ehollindale3r@guardian.co.uk,F,29.0,60.590,-36.508738,-54.281149,None,2017-10-05,True,2018-04-12 15:45:51,"Grytviken, China",entrée/plat/dessert,Esmeralda Hollindale,China,None
10,27446,Skippie,Arlott,sarlottg@so-net.ne.jp,M,49.0,27.797,38.954230,55.877840,None,2017-08-13,True,2018-10-06 10:55:59,"Malaya Dubna, Russia",entrée/plat/dessert,Skippie Arlott,Russia,None
11,27550,Shermie,Locard,slocard3c@lycos.com,M,65.0,35.948,112.813530,-7.305958,None,2018-09-29,False,2019-05-03 00:23:45,"Wonorejo, Indonesia",entrée/plat/dessert,Shermie Locard,Indonesia,None


### Filtrer les numéros de téléphone invalides

In [88]:
ok_phone = df_clean.phone.str.contains(r"^(\d{10})$",na=False)
df_clean = df_clean[ok_phone]

/home/xavier/.local/lib/python3.6/site-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [89]:
df_clean.head(10)

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country,is_mobile_phone
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.180,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Czech Republic,None
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.260,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Czech Republic,None
12,27548,Stanley,McCullen,smccullen3a@timesonline.co.uk,M,60.0,NaN,119.141473,28.074649,0875503094,2017-06-05,True,2017-06-05 00:00:00,"Longquan, China",entrée/plat/dessert,Stanley McCullen,China,None
15,27619,Shelley,Skeemor,sskeemor59@princeton.edu,F,44.0,41.500,111.878471,-7.584917,0547556034,2019-01-24,False,2019-05-04 13:18:24,"Guyangan, Indonesia",entrée/plat/dessert,Shelley Skeemor,Indonesia,None
16,27442,Mort,Midford,mmidfordc@yale.edu,M,59.0,NaN,6.848501,5.959253,0811321968,2019-06-25,False,2019-06-25 00:00:00,"Ozubulu, Nigeria",entrée/plat,Mort Midford,Nigeria,None
20,27462,Graig,Brownsea,gbrownseaw@arstechnica.com,M,54.0,80.230,110.393439,-7.213328,0262855559,2018-05-22,False,2019-09-28 13:28:10,"Sidomukti, Indonesia",entrée/plat/dessert,Graig Brownsea,Indonesia,None
24,27563,Andre,Lugsdin,alugsdin3p@microsoft.com,M,30.0,40.630,12.219712,57.121813,0144026028,2017-01-15,False,2019-06-03 02:11:38,"Varberg, Sweden",entrée/plat/dessert,Andre Lugsdin,Sweden,None
27,27551,Brigg,Bryett,bbryett3d@prlog.org,M,20.0,28.050,102.638296,24.301245,0746863267,2017-10-25,False,2019-04-10 01:46:26,"Dayingjie, China",entrée/plat/dessert,Brigg Bryett,China,"<_sre.SRE_Match object; span=(0, 2), match='07'>"
30,27440,Erick,Meharry,emeharrya@wikispaces.com,M,52.0,44.792,3.575175,47.794456,0098166909,2017-04-21,True,2018-07-11 08:22:11,"Auxerre, France",entrée/plat/dessert,Erick Meharry,France,None
31,27593,Evelyn,Olenichev,eolenichev4j@tripadvisor.com,F,44.0,54.252,23.711756,37.944288,0214068650,2017-01-02,False,2019-08-07 22:59:39,"Néa Smýrni, Greece",entrée/plat/dessert,Evelyn Olenichev,Greece,None


### Ajout d'une colonne indiquant si le téléphone est un portable

In [92]:
df_clean["is_mobile_phone"] = df_clean.phone.str.contains(r"^(06|07)")

In [93]:
df_clean.head(10)

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country,is_mobile_phone
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.180,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Czech Republic,False
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.260,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Czech Republic,False
12,27548,Stanley,McCullen,smccullen3a@timesonline.co.uk,M,60.0,NaN,119.141473,28.074649,0875503094,2017-06-05,True,2017-06-05 00:00:00,"Longquan, China",entrée/plat/dessert,Stanley McCullen,China,False
15,27619,Shelley,Skeemor,sskeemor59@princeton.edu,F,44.0,41.500,111.878471,-7.584917,0547556034,2019-01-24,False,2019-05-04 13:18:24,"Guyangan, Indonesia",entrée/plat/dessert,Shelley Skeemor,Indonesia,False
16,27442,Mort,Midford,mmidfordc@yale.edu,M,59.0,NaN,6.848501,5.959253,0811321968,2019-06-25,False,2019-06-25 00:00:00,"Ozubulu, Nigeria",entrée/plat,Mort Midford,Nigeria,False
20,27462,Graig,Brownsea,gbrownseaw@arstechnica.com,M,54.0,80.230,110.393439,-7.213328,0262855559,2018-05-22,False,2019-09-28 13:28:10,"Sidomukti, Indonesia",entrée/plat/dessert,Graig Brownsea,Indonesia,False
24,27563,Andre,Lugsdin,alugsdin3p@microsoft.com,M,30.0,40.630,12.219712,57.121813,0144026028,2017-01-15,False,2019-06-03 02:11:38,"Varberg, Sweden",entrée/plat/dessert,Andre Lugsdin,Sweden,False
27,27551,Brigg,Bryett,bbryett3d@prlog.org,M,20.0,28.050,102.638296,24.301245,0746863267,2017-10-25,False,2019-04-10 01:46:26,"Dayingjie, China",entrée/plat/dessert,Brigg Bryett,China,True
30,27440,Erick,Meharry,emeharrya@wikispaces.com,M,52.0,44.792,3.575175,47.794456,0098166909,2017-04-21,True,2018-07-11 08:22:11,"Auxerre, France",entrée/plat/dessert,Erick Meharry,France,False
31,27593,Evelyn,Olenichev,eolenichev4j@tripadvisor.com,F,44.0,54.252,23.711756,37.944288,0214068650,2017-01-02,False,2019-08-07 22:59:39,"Néa Smýrni, Greece",entrée/plat/dessert,Evelyn Olenichev,Greece,False
